In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from scipy.ndimage import distance_transform_edt, label
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.morphology import remove_small_objects
from tifffile import imread, imwrite
from skimage.measure import regionprops_table
import pandas as pd
import imageio

# ==== Load Image ====
img = "C://Users//nahue//Downloads//Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.tif"
volume_raw = imread(img)
print("Raw shape:", volume_raw.shape)

# Select first channel
volume = volume_raw[:, 0, :, :]
print("Volume shape:", volume.shape)

# ==== Preprocess ====
volume = (volume - volume.min()) / (volume.max() - volume.min())
thresh = threshold_otsu(volume)
binary = volume > thresh
binary = remove_small_objects(binary, min_size=200)

# ==== Segmentation ====
distance = distance_transform_edt(binary)
coords = peak_local_max(distance, labels=binary, footprint=np.ones((3, 3, 3)), exclude_border=False)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = label(mask)
labels_ws = watershed(-distance, markers, mask=binary)

# ==== Visualize 6 Slices ====
fig, axes = plt.subplots(3, 3, figsize=(20,12))#2,3
step = max(1, volume.shape[0] // 9)#6
for i, ax in enumerate(axes.flat):
    z = i * step
    ax.imshow(labels_ws[z], cmap="nipy_spectral")
    ax.set_title(f"Z = {z}")
    ax.axis("off")
plt.tight_layout()
plt.show()

# ==== Export segmentation as TIF ====
imwrite("segmentation_labels.tif", labels_ws.astype(np.uint16))
imwrite("intensity_volume.tif", (volume * 255).astype(np.uint8))

# ==== Export animated GIF ====
frames = [(labels_ws[z] * 10).astype(np.uint8) for z in range(volume.shape[0])]
imageio.mimsave("segmentation_stack.gif", frames, duration=0.1)

# ==== Export measurements ====
props = regionprops_table(labels_ws, properties=['label', 'area', 'centroid'])
df = pd.DataFrame(props)
df.to_csv("segmentation_measurements.csv", index=False)
#print("Exported segmentation and region data.")
#print(df.head())